In [ ]:
pip install boto3

In [ ]:
import boto3
import json
import getpass
import os

# Set constants

# Obtain the Cognito identity pool ID from the Unity Common Services Team
IDENTITY_POOL_ID = '<IDENTITY_POOL_ID>'

# You AWS Account ID
AWS_ACCOUNT_ID = '<AWS_ACCOUNT_ID>'

# Obtain the Cognito user pool ID from the Unity Common Services Team
COGNITO_USER_POOL_ID = '<COGNITO_USER_POOL_ID>'

# Obtain the Cognito Client ID relevant to your usecase from the Unity Common Services Team
COGNITO_CLIENT_ID = '<COGNITO_CLIENT_ID>'

# AWS Region
REGION = 'us-west-2'

In [ ]:
'''
The following code obtained an ID token using the USER_PASSWORD_AUTH auth flow of client_idp.initiate_auth(). This code interactively requests for
username and password to obtain the ID token. This code is only added for demonstration purposes and usually the Cognito ID token should be available 
as the UNITY_COGNITO_ID_TOKEN environment variable, which is stored during the JupyterLab spawn or an access token refresh.

'''

# Create Cognito IDP client
client_idp = boto3.client('cognito-idp', region_name=REGION)
 
# Promt the user to enter the username and password
username = input('Enter your Cognito username: ')
password = getpass.getpass('Enter your Cognito password: ')
auth_params = {
    "USERNAME" : username,
    "PASSWORD" : password
}

# Get tokens from Cognito
response = client_idp.initiate_auth(
    AuthFlow = 'USER_PASSWORD_AUTH',
    AuthParameters = auth_params,
    ClientId = COGNITO_CLIENT_ID
)

# Read ID token
id_token = response['AuthenticationResult']['IdToken']
print('ID Token = ' + id_token)
os.environ["UNITY_COGNITO_ID_TOKEN"] = str(id_token)

        


In [ ]:
def get_aws_creds_from_cognito_id_token(cognito_identity_pool_id, cognito_user_pool_id, aws_account_id, region, cognito_id_token):
    
    '''
    Stores AWS Access Key ID, Secret Key and Session Token as environment variables UNITY_AWS_ACCESS_KEY_ID, UNITY_AWS_SECRET_ACCESS_KEY and UNITY_AWS_SESSION_TOKEN.
    
    The following parameters should be provided:
    
    Parameter cognito_identity_pool_id: Obtain the Cognito identity pool ID from the Unity Common Services Team
    Parameter cognito_user_pool_id: Obtain the Cognito user pool ID from the Unity Common Services Team
    Parameter aws_account_id: AWS Account ID
    Parameter region: AWS region
    Parameter cognito_id_token: The =Cognito ID token (stored as environment variable UNITY_COGNITO_ID_TOKEN as a result of a previous step/process)

    
    '''
    
    # Create Cognito Identity client
    client_identify = boto3.client('cognito-identity', region_name = REGION)

    id_token = os.environ.get('UNITY_COGNITO_ID_TOKEN')

    # Get Identify ID
    response_identity_get_id = client_identify.get_id(
                AccountId = AWS_ACCOUNT_ID,
                IdentityPoolId = IDENTITY_POOL_ID,
                Logins = {
                    'cognito-idp.us-west-2.amazonaws.com/' + COGNITO_USER_POOL_ID : id_token
                }
    ) 
    IDENTITY_ID = response_identity_get_id['IdentityId']                                                                     


    # Get temporary AWS credentials for the identity    
    aws_credentials = client_identify.get_credentials_for_identity(
        IdentityId = IDENTITY_ID,
        Logins = {
            'cognito-idp.us-west-2.amazonaws.com/' + COGNITO_USER_POOL_ID : id_token
        }
    )

    access_key_id = aws_credentials['Credentials']['AccessKeyId']
    secret_key = aws_credentials['Credentials']['SecretKey']
    session_token = aws_credentials['Credentials']['SessionToken']

    os.environ["UNITY_AWS_ACCESS_KEY_ID"] = str(access_key_id)
    os.environ["UNITY_AWS_SECRET_ACCESS_KEY"] = str(secret_key)
    os.environ["UNITY_AWS_SESSION_TOKEN"] = str(session_token)
    
    print('Temporary AWS Credentials are stored as environment variables: UNITY_AWS_ACCESS_KEY_ID, UNITY_AWS_SECRET_ACCESS_KEY and UNITY_AWS_SESSION_TOKEN')


In [ ]:
id_token = os.environ.get('UNITY_COGNITO_ID_TOKEN')
get_aws_creds_from_cognito_id_token(IDENTITY_POOL_ID, COGNITO_USER_POOL_ID, AWS_ACCOUNT_ID, REGION, id_token)

In [ ]:
print(os.environ.get('UNITY_AWS_ACCESS_KEY_ID'))
print(os.environ.get('UNITY_AWS_SECRET_ACCESS_KEY'))
print(os.environ.get('UNITY_AWS_SESSION_TOKEN'))

In [ ]:
# Create an S3 client
s3_client = boto3.client('s3', 
                      aws_access_key_id = os.environ.get('UNITY_AWS_ACCESS_KEY_ID'),
                      aws_secret_access_key = os.environ.get('UNITY_AWS_SECRET_ACCESS_KEY'),
                      aws_session_token = os.environ.get('UNITY_AWS_SESSION_TOKEN')
                      )
    
s3_response = s3_client.list_buckets()

# Print S3 bucket names
print('Existing buckets:')
for bucket in s3_response['Buckets']:
    print(f'  {bucket["Name"]}')